<a href="https://colab.research.google.com/github/bru5no5/Quizz-Educacional...Videogames-na-Medicina/blob/main/ExecutandoCUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Colab

O ambiente do Google Colab já vem com o suporte CUDA instalado, o que inclui o compilador `nvcc`. Desta forma, não é necessário instalá-lo.

### Ambiente de Execução

Para que seja possível executar os códigos, você deve estar utilizando o ambiente de execução com **GPU**.

In [1]:
!apt search cuda

Sorting... Done
Full Text Search... Done
bart-cuda/jammy 0.7.00-5 amd64
  tools for computational magnetic resonance imaging

boinc-client-nvidia-cuda/jammy 7.18.1+dfsg-4 amd64
  metapackage for CUDA-savvy BOINC client and manager

cublasmp-cuda-11/unknown 0.2.0.385-1 amd64
  NVIDIA cuBLASMp for CUDA 11

cublasmp-cuda-12/unknown 0.2.0.385-1 amd64
  NVIDIA cuBLASMp for CUDA 12

cuda/unknown 12.4.1-1 amd64
  CUDA meta-package

cuda-11-7/unknown 11.7.1-1 amd64
  CUDA 11.7 meta-package

cuda-11-8/unknown 11.8.0-1 amd64
  CUDA 11.8 meta-package

cuda-12-0/unknown 12.0.1-1 amd64
  CUDA 12.0 meta-package

cuda-12-1/unknown 12.1.1-1 amd64
  CUDA 12.1 meta-package

cuda-12-2/unknown 12.2.2-1 amd64
  CUDA 12.2 meta-package

cuda-12-3/unknown 12.3.2-1 amd64
  CUDA 12.3 meta-package

cuda-12-4/unknown 12.4.1-1 amd64
  CUDA 12.4 meta-package

cuda-cccl-11-7/unknown 11.7.91-1 amd64
  CUDA CCCL

cuda-cccl-11-8/unknown 11.8.89-1 amd64
  CUDA CCCL

cuda-cccl-12-0/unknown 12.0.140-1 amd64
  CUDA CCCL

c

In [2]:
!nvidia-smi

Fri May 17 19:06:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [4]:
%%writefile add.cpp
#include <iostream>
#include <math.h>

// function to add the elements of two arrays
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20; // 1M elements

  float *x = new float[N];
  float *y = new float[N];

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the CPU
  add(N, x, y);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  delete [] x;
  delete [] y;

  return 0;
}

Writing add.cpp


In [5]:
!ls -l

total 8
-rw-r--r-- 1 root root  731 May 17 19:06 add.cpp
drwxr-xr-x 1 root root 4096 May 16 13:24 sample_data


In [6]:
!nvcc -o add add.cpp

In [7]:
!time ./add

Max error: 0

real	0m0.023s
user	0m0.018s
sys	0m0.003s


In [8]:
%%writefile add.cu
#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Writing add.cu


In [9]:
!ls -l

total 956
-rwxr-xr-x 1 root root 962600 May 17 19:07 add
-rw-r--r-- 1 root root    731 May 17 19:06 add.cpp
-rw-r--r-- 1 root root    930 May 17 19:07 add.cu
drwxr-xr-x 1 root root   4096 May 16 13:24 sample_data


In [10]:
!nvcc -o add_cuda add.cu

In [11]:
!nvprof ./add_cuda

==1545== NVPROF is profiling process 1545, command: ./add_cuda
Max error: 0
==1545== Profiling application: ./add_cuda
==1545== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  109.53ms         1  109.53ms  109.53ms  109.53ms  add(int, float*, float*)
      API calls:   39.34%  149.46ms         2  74.730ms  38.505us  149.42ms  cudaMallocManaged
                   31.61%  120.09ms         1  120.09ms  120.09ms  120.09ms  cudaLaunchKernel
                   28.84%  109.54ms         1  109.54ms  109.54ms  109.54ms  cudaDeviceSynchronize
                    0.14%  532.46us         2  266.23us  258.15us  274.30us  cudaFree
                    0.06%  220.23us       114  1.9310us     178ns  102.88us  cuDeviceGetAttribute
                    0.00%  13.493us         1  13.493us  13.493us  13.493us  cuDeviceGetName
                    0.00%  10.285us         1  10.285us  10.285us  10.285us  cuDeviceGetPCIBusId
        

In [12]:
!time ./add_cuda

Max error: 0

real	0m0.256s
user	0m0.131s
sys	0m0.117s


In [13]:
%%writefile add_block.cu
#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 256>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Writing add_block.cu


In [14]:
!nvcc -o add_block add_block.cu

In [15]:
!nvprof ./add_block

==1675== NVPROF is profiling process 1675, command: ./add_block
Max error: 0
==1675== Profiling application: ./add_block
==1675== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.9478ms         1  3.9478ms  3.9478ms  3.9478ms  add(int, float*, float*)
      API calls:   70.62%  121.01ms         2  60.504ms  59.007us  120.95ms  cudaMallocManaged
                   26.66%  45.688ms         1  45.688ms  45.688ms  45.688ms  cudaLaunchKernel
                    2.31%  3.9583ms         1  3.9583ms  3.9583ms  3.9583ms  cudaDeviceSynchronize
                    0.30%  512.32us         2  256.16us  235.55us  276.77us  cudaFree
                    0.10%  164.09us       114  1.4390us     167ns  64.572us  cuDeviceGetAttribute
                    0.01%  13.250us         1  13.250us  13.250us  13.250us  cuDeviceGetName
                    0.00%  6.6270us         1  6.6270us  6.6270us  6.6270us  cuDeviceGetPCIBusId
      

In [16]:
!time ./add_block

Max error: 0

real	0m0.157s
user	0m0.026s
sys	0m0.124s


In [17]:
!time ./add_cuda

Max error: 0

real	0m0.267s
user	0m0.129s
sys	0m0.130s


In [18]:
%%writefile add2.cu
#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Writing add2.cu


In [19]:
!nvcc -o add2 add2.cu

In [20]:
!nvprof ./add2

==1847== NVPROF is profiling process 1847, command: ./add2
Max error: 0
==1847== Profiling application: ./add2
==1847== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  134.96ms         1  134.96ms  134.96ms  134.96ms  add(int, float*, float*)
      API calls:   56.09%  134.98ms         1  134.98ms  134.98ms  134.98ms  cudaDeviceSynchronize
                   43.51%  104.69ms         2  52.345ms  54.570us  104.64ms  cudaMallocManaged
                    0.22%  538.95us         2  269.47us  248.26us  290.69us  cudaFree
                    0.11%  255.48us         1  255.48us  255.48us  255.48us  cudaLaunchKernel
                    0.06%  140.04us       114  1.2280us     138ns  56.604us  cuDeviceGetAttribute
                    0.00%  11.695us         1  11.695us  11.695us  11.695us  cuDeviceGetName
                    0.00%  5.5870us         1  5.5870us  5.5870us  5.5870us  cuDeviceGetPCIBusId
                

In [21]:
!time ./add2

Max error: 0

real	0m0.290s
user	0m0.166s
sys	0m0.120s


In [22]:
%%writefile multiply.cu
#include <iostream>
#include <math.h>
// Kernel function to multiply the elements of two arrays
__global__
void multiply(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] * y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  multiply<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 2.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-2.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Writing multiply.cu


In [23]:
!nvcc -o multiply multiply.cu

In [24]:
!nvprof ./multiply

==2307== NVPROF is profiling process 2307, command: ./multiply
Max error: 0
==2307== Profiling application: ./multiply
==2307== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  139.27ms         1  139.27ms  139.27ms  139.27ms  multiply(int, float*, float*)
      API calls:   45.90%  139.28ms         1  139.28ms  139.28ms  139.28ms  cudaDeviceSynchronize
                   39.72%  120.54ms         2  60.270ms  67.495us  120.47ms  cudaMallocManaged
                   14.14%  42.922ms         1  42.922ms  42.922ms  42.922ms  cudaLaunchKernel
                    0.18%  544.94us         2  272.47us  271.21us  273.73us  cudaFree
                    0.05%  150.76us       114  1.3220us     152ns  58.635us  cuDeviceGetAttribute
                    0.00%  12.388us         1  12.388us  12.388us  12.388us  cuDeviceGetName
                    0.00%  6.8020us         1  6.8020us  6.8020us  6.8020us  cuDeviceGetPCIBusId
   

In [25]:
!time ./multiply

Max error: 0

real	0m0.307s
user	0m0.162s
sys	0m0.129s
